In [1]:
import cv2
import numpy as np

ww =120
hh = 120
offset = 6
y1 = 550

#FPS to video
delay = 60

detect = []
carros = 0 


def pega_center(x,y,w,h): #this function to find the center of the rectangele
                        #on the vehicle
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x+x1
    cy = y+y1
    return cx,cy

cap = cv2.VideoCapture("Data/highway traffic.mp4")
#cap = cv2.resize(cap, (550,550))
BGS = cv2.createBackgroundSubtractorMOG2()

while True:
    ret, frame1 = cap.read()
    #print(frame1.shape)
    #tempo = float(1/delay)
    #sleep(tempo)
    
    gray = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    blur = cv2.GaussianBlur(gray, (3,3),5)
    img_sub = BGS.apply(blur)
    dilate = cv2.dilate(img_sub, np.ones((5,5)))
    
    contour,_ = cv2.findContours(dilate, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    cv2.line(frame1, (25,y1), (1200,y1), (176,130,39), 2)
    
    
    for (i,c) in enumerate(contour):
        (x,y,w,h) = cv2.boundingRect(c)
        
        validar_contour = (w>=ww) and (h>=hh) 
        #هنا مثلا بدي اسوي ديتيكت للسيارات القريبة ويرسم عليها مربع
        #اما السيارات البعيدة مايرسم عليها مربع
        if not validar_contour:
            continue
            
        cv2.rectangle(frame1, (x,y), (x+w,y+h), (0,255,0), 2)
        center = pega_center(x,y,w,h)
        detect.append(center)
        cv2.circle(frame1, center, 4,(0,0,255), -1)
        
        for (x,y) in detect:
            if (y<(y1+offset)) and (y>(y1-offset)):
                carros += 1
                cv2.line(frame1, (25,y1), (1200, y1), (0,127,255), 3)
                detect.remove((x,y))
                print("No. of Cars detected:" + str(carros))
                
    cv2.putText(frame1, "Vehicle Count" + str(carros), (320,70), cv2.FONT_HERSHEY_COMPLEX_SMALL,1,(0,255,0))
    cv2.imshow("Original Video", frame1)
    cv2.imshow("Dilate", dilate)
    cv2.imshow("img_sub", img_sub)
    
    if cv2.waitKey(2) == 27:
        break
        
cv2.destroyAllWindows()
cap.release()

No. of Cars detected:1
No. of Cars detected:2
No. of Cars detected:3
